In [2]:
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_bbdd as sb
from src import dicc_query_creacion as dq
import psycopg2
from dotenv import load_dotenv
import os
load_dotenv()
import pymongo
import numpy as np
import pandas as pd





In [3]:
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']
actores = imdb['Actores']
api = imdb['Api']

In [190]:
postres = sb.postgres('StreamWise_1', "postgres", 'admin', 'localhost', '5432')

In [191]:
for query in dq.dicc_query_creacion.values():
    postres.query_creacion(query)

# Genero

In [79]:
generos = set()
for peli in list(api.find()):
    for genero in peli['genre']:
        generos.add(genero)
list_generos = []
for i, genero in enumerate(list(generos), start=1):
    list_generos.append((i,genero))

In [85]:
df_gen = pd.DataFrame(list_generos, columns =['id_genero','nombre'])

# Tipos

In [92]:
tipos = set()
for peli in list(api.find()):
    tipos.add(peli['type'])
list_tipos = []
for i, tipo in enumerate(list(tipos), start=1):
    list_tipos.append((i,tipo))

In [93]:
df_tipos = pd.DataFrame(list_tipos, columns =['id_tipo', 'nombre'])

In [9]:
postres.query_insercion(dq.dicc_query_insercion['genero'], postres.lista_tuplas_peque(generos))

27 registros insertados


# PELIS

In [10]:
lista_peliculas = []
for peli in tqdm(list(peliculas.find())):
    imdb = peli['id_IMDB']
    titulo = api.find_one({'id':peli['id_IMDB']})['title']
    year = api.find_one({'id':peli['id_IMDB']})['year']
    genero = api.find_one({'id':peli['id_IMDB']})['genre']
    tipo = api.find_one({'id':peli['id_IMDB']})['type']
    puntuacion = peli['puntuacion']
    duracion = peli['duracion']

    tupla = (imdb,titulo, year, genero, tipo, puntuacion, duracion)

    lista_peliculas.append(tupla)

100%|██████████| 31487/31487 [1:00:01<00:00,  8.74it/s]


In [214]:
df_pelis =  pd.DataFrame(lista_peliculas, columns =['id','titulo', 'año', 'genero','tipo','puntuacion', 'duracion'])


In [215]:
df_pelis

,id,titulo,año,genero,tipo,puntuacion,duracion
0,tt0102921,Si da jia zu zhi long hu xiong di,1991,[Action],Movie,6.7,116.0
1,tt0102593,One Good Cop,1991,[Action],Movie,5.8,114.0
2,tt0102844,Saudagar,1991,[Action],Movie,6.7,213.0
3,tt0102814,Sa Tam Chai yue Chow Si Lai,1991,[Action],Movie,5.3,89.0
4,tt0102655,The Perfect Weapon,1991,[Action],Movie,6.0,112.0
...,...,...,...,...,...,...,...
31482,tt12058584,Schachnovelle,2021,[War],Movie,6.8,110.0
31483,tt12199174,Pan ni zhe,2021,[War],TV Series,8.3,45.0
31484,tt10013998,Episode #1.8,2012,[Drama],TV Episode,NaN,NaN
31485,tt10014002,Episode #1.9,2012,[Drama],TV Episode,NaN,NaN


In [14]:
df_pelis.to_csv('../data/df_pelis.csv')

In [57]:
postres.query_insercion(dq.dicc_query_insercion['peli'], lista_peliculas)

29062 registros insertados


# Personas
- actores
- directores
- Guionistas

In [16]:
lista_personas = []
for peli in tqdm(list(peliculas.find())):
    for director in peli['direccion']:
        if actores.find_one({'nombre':director}):
            if  type(actores.find_one({'nombre':director})['roles']) is float:
                continue
            elif  type(actores.find_one({'nombre':director})['roles']) is not list:
                continue
            elif 'Dirección' in actores.find_one({'nombre':director})['roles']:
                continue
            else:
                actores.update_one({'nombre':director},{'$set': {'roles': actores.find_one({'nombre':director})['roles'].append('Dirección')}})
            
        else:
            nombre = director
            año = np.nan
            conocido = list(pd.DataFrame(peliculas.find({'direccion': director},{'id_IMDB':1, '_id':0}))['id_IMDB'])
            roles = ['Dirección']
            premios = np.nan
            nominaciones = np.nan
            tupla = (nombre,año,conocido, roles, premios, nominaciones)
            lista_personas.append(tupla)
                
    for guionista in peli['guion']:
        if actores.find_one({'nombre':guionista}):
            if  type(actores.find_one({'nombre':guionista})['roles']) is float:
                continue
            elif  type(actores.find_one({'nombre':guionista})['roles']) is not list:
                continue
            elif 'Guion' in actores.find_one({'nombre':guionista})['roles'] :
                continue
            else:
                actores.update_one({'nombre':guionista},{'$set': {'roles': actores.find_one({'nombre':guionista})['roles'].append('Guion')}})


        else:
            nombre = guionista
            año = np.nan
            conocido = list(pd.DataFrame(peliculas.find({'guion': guionista},{'id_IMDB':1, '_id':0}))['id_IMDB'])
            roles = ['Guion']
            premios = np.nan
            nominaciones = np.nan
            tupla = (nombre,año,conocido, roles, premios, nominaciones)
            lista_personas.append(tupla)

for actor in tqdm(list(actores.find())):
    nombre = actor['nombre']
    año = actor['year']
    conocido = actor['conocido']
    roles = actor['roles']
    premios = actor['premios']
    try:
        nominaciones = actor['nominaciones']
    except:
        nominaciones = np.nan
    tupla = (nombre,año,conocido, roles, premios, nominaciones)
    lista_personas.append(tupla)


100%|██████████| 8930/8930 [00:00<00:00, 1130106.95it/s]


In [18]:
df_personas = pd.DataFrame(lista_personas, columns=['nombre', 'año', 'conocido', 'roles', 'premios', 'nominaciones'])


# Roles

In [120]:
roless = set()
for roles in df_personas['roles']:
    if type(roles) is not list:
        continue
    for rol in roles:
        if any(char.isdigit() for char in rol):
            continue
        roless.add(rol)

list_roles = []
for i, rol in enumerate(list(roless), start=1):
    list_roles.append((i,rol))


In [122]:
df_roles = pd.DataFrame(list_roles, columns =['id_rol','nombre'])

# Relaciones

In [230]:
df_relaciones = df_pelis[['id', 'genero', 'tipo']].merge(df_tipos, how = 'inner', left_on='tipo', right_on='nombre')

In [231]:
df_relaciones.drop(['tipo','nombre'], axis = 1, inplace=True)

In [232]:
df_relaciones = df_relaciones.explode('genero')

In [233]:
df_relaciones = df_relaciones.merge(df_gen, how='inner',left_on='genero', right_on='nombre')

In [234]:
df_relaciones.drop(['nombre', 'genero'], axis = 1, inplace=True)

In [235]:
df_personas['id_persona'] = df_personas.reset_index().index + 1

In [236]:
df_personas = df_personas[['id_persona'] + [col for col in df_personas.columns if col != 'id_persona']]

In [238]:
df_personas_explode = df_personas.explode('conocido').explode('roles').drop(['nombre', 'año', 'premios', 'nominaciones'], axis = 1)

In [239]:
df_personas_explode = df_personas_explode.merge(df_roles, how='inner', left_on='roles', right_on='nombre').drop(['roles', 'nombre'], axis = 1)

In [240]:
df_relaciones = df_relaciones.merge(df_personas_explode, how='inner', left_on='id', right_on='conocido').drop('conocido', axis =1)

In [241]:
df_relaciones.rename(columns = {'id':'id_pelicula'},inplace=True)

# Inserción

In [196]:
postres.query_insercion('genero', df_gen)

[[1, 'Adult'], [2, 'Comedy'], [3, 'War'], [4, 'Music'], [5, 'Sci-Fi'], [6, 'Reality-TV'], [7, 'Fantasy'], [8, 'Romance'], [9, 'Family'], [10, 'Crime'], [11, 'Sport'], [12, 'Thriller'], [13, 'Action'], [14, 'Musical'], [15, 'Biography'], [16, 'Animation'], [17, 'Talk-Show'], [18, 'News'], [19, 'Game-Show'], [20, 'Drama'], [21, 'Documentary'], [22, 'Adventure'], [23, 'Western'], [24, 'Short'], [25, 'History'], [26, 'Mystery'], [27, 'Horror']]
INSERT INTO "genero" (id_genero, nombre) VALUES (%s, %s)
27 registros insertados


In [200]:
postres.query_insercion('tipo', df_tipos)

13 registros insertados


In [202]:
postres.query_insercion('rol', df_roles)


57 registros insertados


In [207]:
df_pelis = df_pelis[['id', 'titulo', 'año', 'puntuacion', 'duracion']]
df_pelis.rename(columns={'id':'id_pelicula'},inplace=True)

In [211]:
postres.query_insercion('pelicula', df_pelis)

31487 registros insertados


In [245]:
df_personas = df_personas[['id_persona', 'nombre', 'año', 'premios', 'nominaciones']]

In [250]:
postres.query_insercion('persona', df_personas)

38577 registros insertados


In [251]:
df_relaciones

,id_pelicula,id_tipo,id_genero,id_persona,id_rol
0,tt0102921,12,13,1,19
1,tt0102921,12,13,2,18
2,tt0102921,12,13,3,18
3,tt0102921,12,13,270,18
4,tt0102921,12,13,282,18
...,...,...,...,...,...
65468,tt12058584,12,3,29643,18
65469,tt12058584,12,3,29644,18
65470,tt12199174,10,3,29645,19
65471,tt12199174,10,3,29646,19


In [252]:
postres.query_insercion('relaciones', df_relaciones)

65473 registros insertados
